# Radar Data Streaming and Pre-Processing

### This notebook demonstrates how to read the data from the radar using python wrapper for Infineon Radar SDK

#### Importing required libraries 

    - ifxdaq --> is the python wrapper for Radar SDK
    - processing --> is python module for basic processing to get the phase information from the radar signal
    

In [20]:
import ifxdaq
import processing
import numpy as np
#print(ifxdaq.__version__)
from ifxdaq.sensor.radar_ifx import RadarIfxAvian
time_mins = 1
time_secs = time_mins*60
frame_count = time_secs*1000

#### In the folder 'radar_configs' you find radar configuration parameters. It contains radar parameters :

    - transmit power level
    - start/end frequency
    - sampling rate
    - # of chirps per frame
    - # of samples per chirp
    - # of transmit/receive antennas
    - MIMO mode selection

In [21]:
config_file = "radar_configs/RadarIfxBGT60.json"
raw_data    = []

#### the next code block initiliaze the radar with set configuration and loop through the continuous frames; when it  reaches 5000 frames which is 5 second data according to our set configuration, we process it to get the phase information out of it.

In [22]:
with RadarIfxAvian(config_file) as device:                             # Initialize the radar with configurations
    
    for i_frame, frame in enumerate(device):                           # Loop through the frames coming from the radar
        
        raw_data.append(np.squeeze(frame['radar'].data/(4095.0)))      # Dividing by 4095.0 to scale the data
        
        if(len(raw_data) > frame_count-1 and len(raw_data) % frame_count == 0):        # 5000 is the number of frames. which corresponds to 5seconds
            
            data = np.swapaxes(np.asarray(raw_data), 0, 1)

            phases, abses, _, _ = processing.do_processing(data)       # preprocessing to get the phase information
            
            phases              = np.mean(phases, axis=0)
            
            range_data = processing.processing_rangeData(data)
            break

Note: There is a function to get range_data called "processing_rangeData" which will be usefull to detect the movements.

In [23]:
import datetime
datetime_now = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

farness=0
moving=0

# farness = np.array([0]) # 0-near ...
# moving = np.array([0]) # 0 not moving, 1 a little, 2 a lot

filename = "mixed_farness_"+str(farness)+"_moving_"+str(moving)+'_radardata_' + datetime_now + '.npz'
print(filename)
np.savez(filename, abses=abses, phases=phases, range_data=range_data, farness=farness, moving=moving)

mixed_farness_0_moving_0_radardata_2022_05_08_21_47_01.npz
